## Decision Tree

- the Decision Tree algorithm is constructs a Decision Tree from the data that if followed, classifies data correctly.

- To construct the tree, each time, the algorithm splits the data based on a question related to a feature. The goal is to split the data in the most pure way ( at best, the question should separate data of class c1 from data of class c2 )

- To measure the "impurity" of the split ( a group of rows ), the Metric "Gini Impurity" is used to calculate the "incertainty" of each split ( 0 => Data is homogeneous, ie. pure, 1 => Data contains equal numbers of each class, ie. impure )

- $ Gini = 1- \sum_i^C {p_i}^2 $ where $p_i = \frac{nbr-elements-in-class-i}{nbr-all-elements}$

( nice explanation of the origin of the formula here: https://www.quora.com/What-is-the-interpretation-and-intuitive-explanation-of-Gini-impurity-in-decision-trees )




- Source: https://scikit-learn.org/stable/modules/tree.html

> Some advantages of decision trees are:

>        Simple to understand and to interpret. Trees can be visualised.
>        Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed. Note however that this module does not support missing values.
>        The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.
>        Able to handle both numerical and categorical data. Other techniques are usually specialised in analysing datasets that have only one type of variable. See algorithms for more information.
>        Able to handle multi-output problems.
>        Uses a white box model. If a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model (e.g., in an artificial neural network), results may be more difficult to interpret.
>        Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.
>        Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.

> The disadvantages of decision trees include:

>        Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning (not currently supported), setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.
>        Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.
>        The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in an ensemble learner, where the features and samples are randomly sampled with replacement.
>        There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.
>        Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.



## Imports

In [1]:
import pandas as pd
import numpy as np
from math import pow

## Load Dataset "Tennis"

In [2]:
df = pd.read_csv('data/tennis.csv')
nbr_rows = len(df)
features = df.columns[:-1]
classes = set(df['play'])
df

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [3]:

def calculate_gini(rows):
    s = 0
    for c in classes:
        s += pow(len(rows[rows['play'] == c]) / len(rows), 2)
    gini = 1 - s
    return gini

def calculate_info_gain(rows, left, right):
    gini_root = calculate_gini(rows)
    gini_left = calculate_gini(left)
    gini_right = calculate_gini(right)
    return gini_root - ((len(left)/len(rows))*gini_left + (len(right)/len(rows))*gini_right)

def split(rows, feature, value):
    # TODO: condition if numeric 
    if type(value) == int or type(value) == bool:
        right_condition = rows[feature] >= value
        left_condition = rows[feature] < value
    else:
        right_condition = rows[feature] == value
        left_condition = rows[feature] != value
    right = rows[right_condition]
    left = rows[left_condition]
    return left, right

def best_split(rows):
    max_info_gain = -1
    ret = (None, None, None, None) # (feature, value, left, right)
    #print('Searching best split ...')
    for f in features:
        for v in set(rows[f]):
            #print(f"Splitting on {f} / {v}")
            left, right = split(rows, f, v)
            if len(left) == 0 or len(right) == 0:
                #print("no split")
                continue
            info_gain = calculate_info_gain(rows, left, right)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                ret = (f, v, left, right)
    return ret


In [4]:
class Leaf:
    def __init__(self, rows):
        self.rows = rows
        self.class_counts = {}
        for i, r in rows.iterrows():
            try:
                cl = r['play']
                self.class_counts[cl] += 1
            except:
                self.class_counts[cl] = 1
    def __str__(self):
        s = ""
        for cl in self.class_counts:
            s += "{} tennis => {} | \n".format(cl, self.class_counts[cl])
        return s

    def __repr__(self):
        return str(self)

class Node:
    def __init__(self, feature, value, left, right):
        self.feature = feature
        self.value   = value
        self.left    = left   # might be a Node or a Leaf
        self.right   = right # might be a Node or a Leaf ( right means that it contains rows that are true in regard to the question for the feature)
    
    def __str__(self):
        s = ""
        operator = ""
        if type(self.value) == int or type(self.value) == float:
            operator = ">="
            neg_operator = "<"
        else:
            operator = "is"
            neg_operator = "is not"
        s += f"{self.feature} {operator} {self.value} ?\n\t"
        s += "Yes => \n\t" + str(self.right)
        s += "\n"
        s += f"{self.feature} {neg_operator} {self.value} => \n\t" + str(self.left)
        return s
    
    def __repr__(self):
        return str(self)


In [5]:
#split(df, 'outlook', 'sunny')
#f, v, l, r = best_split(df)

#print(df.iloc[13])
#best_split()
#print(Node(f, v, Leaf(l), Leaf(r)))
#r_str = str(Leaf(r))
# pow(4,2)

In [6]:

def build_tree(rows):
    feature, value, left, right = best_split(rows)
    if left is None and right is None:
        print('Creating Leaf')
        return Leaf(rows)
    else:
        print("Branch {} / {} | Left: {}, Right: {}".format(feature, value, len(left), len(right)))
        r_branch = build_tree(right)
        l_branch = build_tree(left)
        return Node(feature, value, l_branch, r_branch)

In [7]:
t = build_tree(df)

Branch outlook / overcast | Left: 10, Right: 4
Branch temp / mild | Left: 3, Right: 1
Creating Leaf
Branch temp / cool | Left: 2, Right: 1
Creating Leaf
Branch humidity / high | Left: 1, Right: 1
Creating Leaf
Creating Leaf
Branch humidity / high | Left: 5, Right: 5
Branch outlook / sunny | Left: 2, Right: 3
Branch temp / mild | Left: 2, Right: 1
Creating Leaf
Branch windy / False | Left: 1, Right: 1
Creating Leaf
Creating Leaf
Branch windy / False | Left: 1, Right: 1
Creating Leaf
Creating Leaf
Branch windy / False | Left: 2, Right: 3
Branch outlook / sunny | Left: 2, Right: 1
Creating Leaf
Branch temp / mild | Left: 1, Right: 1
Creating Leaf
Creating Leaf
Branch outlook / sunny | Left: 1, Right: 1
Creating Leaf
Creating Leaf


In [8]:
t ## We clearly see that it's overfit ! some features need to be converted to numbers and needs pruning

outlook is overcast ?
	Yes => 
	temp is mild ?
	Yes => 
	yes tennis => 1 | 

temp is not mild => 
	temp is cool ?
	Yes => 
	yes tennis => 1 | 

temp is not cool => 
	humidity is high ?
	Yes => 
	yes tennis => 1 | 

humidity is not high => 
	yes tennis => 1 | 

outlook is not overcast => 
	humidity is high ?
	Yes => 
	outlook is sunny ?
	Yes => 
	temp is mild ?
	Yes => 
	no tennis => 1 | 

temp is not mild => 
	windy is False ?
	Yes => 
	no tennis => 1 | 

windy is not False => 
	no tennis => 1 | 

outlook is not sunny => 
	windy is False ?
	Yes => 
	yes tennis => 1 | 

windy is not False => 
	no tennis => 1 | 

humidity is not high => 
	windy is False ?
	Yes => 
	outlook is sunny ?
	Yes => 
	yes tennis => 1 | 

outlook is not sunny => 
	temp is mild ?
	Yes => 
	yes tennis => 1 | 

temp is not mild => 
	yes tennis => 1 | 

windy is not False => 
	outlook is sunny ?
	Yes => 
	yes tennis => 1 | 

outlook is not sunny => 
	no tennis => 1 | 